<a href="https://colab.research.google.com/github/LuisPeMoraRod/AI-Laboratories/blob/main/Lab5_MatrixProgrammed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# import libraries
import torch


### Ejercicio 1
Implemente la función *calcularTrazaMatriz*, la cual calcula la traza de una matriz usando únicamente operaciones básicas en Pytorch (multiplicación, multiplicación por elemento, matriz identidad, etc), prescindiendo de estructuras de repetición como el for el while. (10 pts) 


### Ejercicio 2
Desarrolle la tranpuesta de una matriz. (10 pts)

In [2]:
import numpy as np


# Método que permite obtener la matriz transpuesta.


def transpose_matrix(matrix):

    # Obtener el numero de filas y columnas de la matriz
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    # Creamos una matriz de ceros invertida
    transpose = [[0 for j in range(num_rows)] for i in range(num_cols)]

    # iteramos cada elemento de la matriz junto con la creada
    for i in range(num_rows):
        for j in range(num_cols):
            # Agregamos los datos correspondientes a la matriz tranpuesta
            transpose[j][i] = matrix[i][j]

    return transpose

# Método que permite imprimir las matrices.

def printMatrix(matrix):
  for row in matrix:
    print(row)


matriz = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

print("------------ Método programado ----------------")
print("Matriz Original")
printMatrix(matriz)
transpose = transpose_matrix(matriz)
print("Matriz Transpuesta")
printMatrix(transpose)

print("------------ Con librería ----------------")

print("Matriz Transpuesta")

npmatriz = np.array(matriz).transpose()
printMatrix(npmatriz)

------------ Método programado ----------------
Matriz Original
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
Matriz Transpuesta
[1, 4, 7]
[2, 5, 8]
[3, 6, 9]
------------ Con librería ----------------
Matriz Transpuesta
[1 4 7]
[2 5 8]
[3 6 9]


### Ejercicio 3
Desarrolle la matriz adjunta de una matriz (10 pts)

### Ejercicio 4
Desarrolle la inversa de una matriz. Debe validar si es invertible o no. (20 pts)

In [3]:
import numpy as np

def determinant(matriz):
    n = len(matriz)
    det = 1
    for i in range(n):
        for j in range(i+1, n):
            if matriz[j][i] != 0:
                coef = matriz[j][i] / matriz[i][i]
                for k in range(i, n):
                    matriz[j][k] -= coef * matriz[i][k]
        det *= matriz[i][i]
    return det




def inverse_matrix(matrix):
 return [[2, -2, 2], [2, 1, 0], [3, -2, 2]]

matriz = [[2, -2, 2], [2, 1, 0], [3, -2, 2]]



print("------------ Método programado ----------------")
print("Matriz Original")
printMatrix(matriz)
inverse = inverse_matrix(matriz)
print("Matriz Inversa")
printMatrix(inverse)

print("------------ Con librería ----------------")

print("Matriz Inversa")

npmatriz = np.array(matriz)

npmatriz_inv = np.linalg.inv(npmatriz)

printMatrix(npmatriz_inv)

------------ Método programado ----------------
 Determinante
-2.0
------------ Método programado ----------------
Matriz Original
[2, -2, 2]
[0.0, 3.0, -2.0]
[0.0, 0.0, -0.33333333333333337]
Matriz Inversa
[2, -2, 2]
[2, 1, 0]
[3, -2, 2]
------------ Con librería ----------------
Matriz Inversa
[0.5        0.33333333 1.        ]
[ 0.          0.33333333 -2.        ]
[-0. -0. -3.]


### Ejercicio 5
La Matriz de covarianza. (50 pts) 